In [1]:
# importing required modules
import pandas as pd
import mysql.connector as msql
import math
import os
import datetime
import re
import requests

In [2]:
#display all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
#Reading She for STEM Incubator file present on source files
directory_path =(r"")
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    data = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from She for STEM - Uttarakhand.csv:


In [4]:
# Create a new dataframe with 'email' column and columns that start with 'Week' 'Video', 'Recording', and 'Master class'
da = data[['Email'] + [col for col in data.columns if col.startswith(('Week','Video', 'SUK', 'Workshop', 'Master Class'))]]


In [5]:
da.shape

(503, 48)

In [6]:
oldcol = list(da.columns)


In [7]:
oldcol = [col for col in oldcol if not col.startswith("Week")]
oldcol = [col.split(" :")[0] for col in oldcol]
oldcol = [col.split(": ")[0] for col in oldcol]

In [8]:
# Initialize variables to keep track of the current week, video count, recording count, and master class count
week_col = None
video_count = 0
recording_count = 0
master_count = 0
workshop_count = 0

# Create an empty list to store the new column names
new_cols = []

# Iterate over each column in the data
for col in data.columns:
    # If the column starts with 'Week'
    if col.startswith('Week'):
        # Split the column name by space and get the second element (the week number)
        week_col = col.split()[1]
        # Reset the video, recording, and master class counts for the new week
        video_count = 0
        recording_count = 0
    # If the column starts with 'Video'
    elif col.startswith('Video'):
        # Increment the video count for the current week
        video_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_V{video_count}')
    # If the column starts with 'Workshop'
    elif col.startswith('Workshop'):
        # Increment the master class count for the current week
        workshop_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_WS{workshop_count}')
    # If the column doesn't start with any of the above prefixes
    # If the column starts with 'Recording'
    elif col.startswith('SUK'):
        # Increment the recording count for the current week
        recording_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_SUK_V')
    # If the column starts with 'Master Class'
    elif col.startswith('Master Class'):
        # Increment the master class count for the current week
        master_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_Master{master_count}')
    # If the column doesn't start with any of the above prefixes
    else:
        # Append the original column name to the list
        new_cols.append(col)

# Remove all columns that start with 'Week' from the data
data = data.loc[:, ~data.columns.str.startswith('Week')]
# Assign the new column names to the data
data.columns = new_cols


In [ ]:
# Reading Incubator_and_attendence_monitoring from Outout files
data =pd.read_csv(r"")

In [11]:
df=pd.DataFrame(data["Email"])

In [12]:
# Get the list of columns starting with 'WK' or 'Email'
selected_columns = [col for col in data.columns if col.startswith('WK') or col.lower() == 'email']


In [13]:
# Keep only the selected columns in the DataFrame
data = data[selected_columns]

In [14]:
rename_col=list(data.columns)

In [15]:
data.columns

Index(['Email', 'WK0_V1', 'WK0_V2', 'WK0_SUK_V', 'WK1_V1', 'WK1_V2', 'WK1_WS1',
       'WK1_Master1', 'WK2_SUK_V', 'WK3_V1', 'WK3_V2', 'WK3_WS2',
       'WK3_Master2', 'WK4_SUK_V', 'WK4_WS3', 'WK5_V1', 'WK5_V2', 'WK5_V3',
       'WK5_WS4', 'WK5_WS5', 'WK5_Master3', 'WK6_WS6', 'WK6_SUK_V', 'WK7_V1',
       'WK7_WS7', 'WK9_V1', 'WK9_V2', 'WK9_WS8', 'WK9_Master4', 'WK10_V1',
       'WK10_V2', 'WK10_V3', 'WK10_V4', 'WK10_WS9', 'WK10_Master5',
       'WK10_WS10', 'WK10_SUK_V'],
      dtype='object')

In [16]:
oldcol

['Email',
 'Video !',
 'Video @',
 'SUK +',
 'Video #',
 'Video $',
 'Workshop @',
 'Master Class #',
 'SUK !',
 'Video %',
 'Video ^',
 'Workshop $',
 'Master Class $',
 'SUK @',
 'Workshop !!',
 'Video $?',
 'Video %+',
 'Video %!',
 'Workshop !#',
 'Workshop %',
 'Master Class !',
 'Workshop ^',
 'SUK #',
 'Video %@',
 'Workshop &',
 'Video %#',
 'Video %$',
 'Workshop *',
 'Master Class %',
 'Video &',
 'Video *',
 'Video ?',
 'Video !+',
 'Workshop ?',
 'Master Class !^',
 'Workshop !@',
 'SUK $']

In [17]:
data.columns = oldcol 

In [18]:
# Function to clean the column names
def clean_column_name(name):
    return name.split(':')[0]

# Apply the function to the column names
data.columns = [clean_column_name(col) for col in data.columns]

In [20]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')

    # If the first part is 'Workshop', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Workshop':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'WS' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col


In [21]:
# Rename the column names using the rename_column function
data.rename(columns=rename_column, inplace=True)

In [ ]:
# Reading Excel file form our source i.e Timestamp_of_Videos
excel_file  = pd.read_excel(r"")

In [24]:
# Iterate over each column in the 'data' DataFrame
for column_name in data.columns:
    # Check if the column exists in the Excel file
    if column_name in excel_file['Code'].values:
        # Get the corresponding value in the "Time" column from the Excel file
        value = excel_file.loc[excel_file['Code'] == column_name, 'Time'].values[0]
        
        # Calculate the percentage value
        percentage_value = (data[column_name] * 100) / (value/3600)
        
        # Assign the calculated percentage value to a new column in the "data1" DataFrame with the same name as the "Code" column
        df[column_name] = percentage_value
    else:
        # Handle the case when the column is present in 'data' but not in the Excel file
        print(f"Column '{column_name}' present in 'data' but not in the Excel file. Skipping.")


Column 'Email' present in 'data' but not in the Excel file. Skipping.


In [26]:
# Renaming column name with old name
df.columns = rename_col 
data.columns = rename_col 

In [27]:
# Changing percent values greater than 100 to 100
def Max_Value(value):
    if value >=100:
        return 100
    else:
        return value

In [28]:
col=df.iloc[:,1:]
df1=col.columns
df[df1]=df[df1].applymap(Max_Value)

C:\Users\spjay\AppData\Local\Temp\ipykernel_4768\2444152655.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[df1]=df[df1].applymap(Max_Value)


# 👇 Define Week Here  👇

In [29]:
end_week = 10

# Weekly Recorded Total Hours

In [30]:
cols_to_select = ['Email']  # Start with 'email' column as the first column
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in data.columns and col != 'email']
    cols_to_select.extend(video_cols)

# Select the desired columns and create a new DataFrame
Weekly = data[cols_to_select].copy()

# Calculate total for each week and add to Weekly dataframe
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in Weekly.columns]
    Weekly[f'WK{week}_Pre_Recorded_Total'] = Weekly[video_cols].sum(axis=1, skipna=True)

# Calculate total for all weeks and add to Weekly dataframe
total_cols = [f'WK{week}_Pre_Recorded_Total' for week in range(end_week + 1) if f'WK{week}_Pre_Recorded_Total' in Weekly.columns]
Weekly['Pre_Recorded_All_Week_Total'] = Weekly[total_cols].sum(axis=1, skipna=True)

# Select the desired columns in the Weekly dataframe
cols_to_select = ['Email'] + total_cols + ['Pre_Recorded_All_Week_Total']
Weekly = Weekly[cols_to_select]


# Recorded Weekly Percentage

In [31]:
cols_to_select = ['Email']  # Start with 'email' column as the first column
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in df.columns and col != 'email']
    cols_to_select.extend(video_cols)

# Select the desired columns and create a new DataFrame
Weekly_Per = df[cols_to_select].copy()

# Calculate total for each week
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in Weekly_Per.columns]
    Weekly_Per[f'Pre_Recorded_WK{week}_Percent'] = Weekly_Per[video_cols].mean(axis=1, skipna=True)

# Calculate total for all weeks
total_cols = [f'Pre_Recorded_WK{week}_Percent' for week in range(end_week + 1) if f'Pre_Recorded_WK{week}_Percent' in Weekly_Per.columns]
Weekly_Per['Pre_Recorded_All_Week_Percent'] = Weekly_Per[total_cols].mean(axis=1, skipna=True)

# Select the desired columns in the Weekly dataframe
cols_to_select = ['Email'] + total_cols + ['Pre_Recorded_All_Week_Percent']
Weekly_Per = Weekly_Per[cols_to_select]


In [32]:
# Merging Dataframe
Weekly = Weekly.merge(Weekly_Per,on= 'Email', how ='left')

# SUK Weekly Total Hours

In [33]:
# Initialize the list of columns to select
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in data.columns:
        cols_to_select.append(col_name)

# Add the 'Email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
SUK_Recorded_Total = data.loc[:, cols_to_select]

# Create new columns with 'SUK' prefix for each week
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    new_col_name = f'SUK{week + 1}'
    if col_name in SUK_Recorded_Total.columns:
        SUK_Recorded_Total[new_col_name] = SUK_Recorded_Total[col_name]

# Drop the original columns (excluding 'Email')
cols_to_drop = [col for col in cols_to_select if col != 'Email']
SUK_Recorded_Total.drop(cols_to_drop, axis=1, inplace=True)

# Select only the columns of float data type for calculating total hours
float_cols = SUK_Recorded_Total.select_dtypes(include='float')

# Calculate the sum of hours for each row and add as a new column
SUK_Recorded_Total['SUK_Total_Hours'] = float_cols.sum(axis=1)




In [34]:
# Merging Dataframe
Weekly = Weekly.merge(SUK_Recorded_Total,on= 'Email', how ='left')

# SUK Average Percentage

In [35]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in df.columns:
        cols_to_select.append(col_name)

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
SUK_Recorded_percent = df.loc[:, cols_to_select]

# Create new columns with 'SUK' prefix for each week
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    new_col_name = f'SUK{week+1}_Percent'
    if col_name in SUK_Recorded_percent.columns:
        SUK_Recorded_percent[new_col_name] = SUK_Recorded_percent[col_name]

# Drop the original columns
SUK_Recorded_percent.drop(cols_to_select[:-1], axis=1, inplace=True)

# Select only the columns of float data type
float_cols = SUK_Recorded_percent.select_dtypes(include='float')

# Calculate the mean of the float columns
SUK_Recorded_percent['SUK_Percentage'] = float_cols.mean(axis=1)


In [36]:
# Merging Dataframe
Weekly = Weekly.merge(SUK_Recorded_percent,on= 'Email', how ='left')

# Weekly Masterclass total hours

In [37]:
# Create a list of all column names up to the end week
cols_to_select = [f'WK{i}_Master{j}' for i in range(end_week + 2) for j in range(1, 10) if f'WK{i}_Master{j}' in data.columns]

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
Masterclass_Total = data.loc[:, cols_to_select]

# Rename columns as "Master{number}"
Masterclass_Total = Masterclass_Total.rename(columns={col_name: f'Master{i+1}' for i, col_name in enumerate(Masterclass_Total.columns) if col_name != 'Email'})

# Create the "Total_Hours" column
Masterclass_Total['Masterclass_Total_Hours'] = Masterclass_Total.select_dtypes(include=['float']).sum(axis=1)


In [38]:
# Merging Dataframe
Weekly = Weekly.merge(Masterclass_Total,on= 'Email', how ='left')

# Masterclass Percentage


In [39]:
# Create a list of all column names up to the end week
cols_to_select = [f'WK{i}_Master{j}' for i in range(end_week + 2) for j in range(1, 10) if f'WK{i}_Master{j}' in df.columns]

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
Masterclass_Per = df.loc[:, cols_to_select]

# Rename columns as "Master{number}"
Masterclass_Per = Masterclass_Per.rename(columns={col_name: f'Master{i +1}_Percent' for i, col_name in enumerate(Masterclass_Per.columns) if col_name != 'Email'})

# Create the "Master_Percentage" column
Masterclass_Per['Master_Percentage'] = Masterclass_Per.select_dtypes(include=['float']).mean(axis=1)


In [40]:
# Merging Dataframe
Weekly = Weekly.merge(Masterclass_Per,on= 'Email', how ='left')

In [41]:
Weekly.columns

Index(['Email', 'WK0_Pre_Recorded_Total', 'WK1_Pre_Recorded_Total',
       'WK2_Pre_Recorded_Total', 'WK3_Pre_Recorded_Total',
       'WK4_Pre_Recorded_Total', 'WK5_Pre_Recorded_Total',
       'WK6_Pre_Recorded_Total', 'WK7_Pre_Recorded_Total',
       'WK8_Pre_Recorded_Total', 'WK9_Pre_Recorded_Total',
       'WK10_Pre_Recorded_Total', 'Pre_Recorded_All_Week_Total',
       'Pre_Recorded_WK0_Percent', 'Pre_Recorded_WK1_Percent',
       'Pre_Recorded_WK2_Percent', 'Pre_Recorded_WK3_Percent',
       'Pre_Recorded_WK4_Percent', 'Pre_Recorded_WK5_Percent',
       'Pre_Recorded_WK6_Percent', 'Pre_Recorded_WK7_Percent',
       'Pre_Recorded_WK8_Percent', 'Pre_Recorded_WK9_Percent',
       'Pre_Recorded_WK10_Percent', 'Pre_Recorded_All_Week_Percent', 'SUK1',
       'SUK3', 'SUK5', 'SUK7', 'SUK11', 'SUK_Total_Hours', 'SUK1_Percent',
       'SUK3_Percent', 'SUK5_Percent', 'SUK7_Percent', 'SUK11_Percent',
       'SUK_Percentage', 'Master1', 'Master2', 'Master3', 'Master4', 'Master5',
       'Mast

# Weekly Workshop total hours

In [43]:
# Create a list of all column names up to the end week
cols_to_select = [f'WK{i}_WS{j}' for i in range(end_week + 1) for j in range(1, 10) if f'WK{i}_WS{j}' in data.columns]

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
Workshop_Total = data.loc[:, cols_to_select]

# Rename columns as "Workshop{number}"
Workshop_Total = Workshop_Total.rename(columns={col_name: f'Workshop{i+1}' for i, col_name in enumerate(Workshop_Total.columns) if col_name != 'Email'})

# Create the "Total_Hours" column
Workshop_Total['Workshop_Total_Hours'] = Workshop_Total.select_dtypes(include=['float']).sum(axis=1)


In [44]:
# Merging Dataframe
Weekly = Weekly.merge(Workshop_Total,on= 'Email', how ='left')

# Workshop Percentage


In [45]:
# Create a list of all column names up to the end week
cols_to_select = [f'WK{i}_WS{j}' for i in range(end_week + 1) for j in range(1, 10) if f'WK{i}_WS{j}' in df.columns]

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
Workshop_Per = df.loc[:, cols_to_select]

# Rename columns as "Master{number}"
Workshop_Per = Workshop_Per.rename(columns={col_name: f'Workshop{i +1}_Percent' for i, col_name in enumerate(Workshop_Per.columns) if col_name != 'Email'})

# Create the "Master_Percentage" column
Workshop_Per['Workshop_Percentage'] = Workshop_Per.select_dtypes(include=['float']).mean(axis=1)


In [46]:
# Merging Dataframe
Weekly = Weekly.merge(Workshop_Per,on= 'Email', how ='left')

In [47]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
Weekly['Email'] = Weekly['Email'].apply(clean_email)

# Adding Genral Info and Exporting

In [ ]:
gi = pd.read_csv(r"")

In [49]:
gi = gi[['Email', 'Name', 'Phone', 'Name_of_College_University','Currently_Pursuing_Degree']].copy()

In [50]:
# Merge with gi
output = pd.merge(gi, Weekly, on='Email', how='outer')

In [51]:
output =output.drop_duplicates()

In [52]:
# Filling Null values with zero and rounding off upto 2 decimal points
output=output.fillna(0)
output=output.round(2)

In [ ]:
# Exporting data to Output files
output.to_csv(r"")

# Storing data to MySQL

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')


In [56]:
cursor =conn.cursor()

In [57]:
# Your existing code for inserting data into the database table
for i, row in Weekly.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(Weekly.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO 09_weekly_attendence ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in Weekly.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [58]:
conn.commit()

# Storing Data in Supabase

In [ ]:
# Supabase URL and API key
url = ''
api_key = ''


# Headers for the request
headers = {
    'apikey': api_key,
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
    'Prefer': 'resolution=merge-duplicates'  # Enable upsert functionality
}


In [60]:
### 09_weekly_attendence

In [61]:
table_name = '09_weekly_attendence'

# Batch size for upserting
batch_size = 1000  # You can adjust this value based on your performance needs

# List to store rows before sending them in batches
batch_data = []

# Iterate through each row in the DataFrame
for i, row in Weekly.iterrows():
    # Replace NaN values with None
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row]
    # Convert row to a dictionary
    row_dict = dict(zip(Weekly.columns, row))
    
    # Add the row to the batch
    batch_data.append(row_dict)
    
    # If the batch size is reached, send the data
    if len(batch_data) >= batch_size:
        # Send a batch of rows
        response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
        
        # Check response
        if response.status_code in [200, 201]:
            print(f'Batch of {len(batch_data)} rows upserted successfully')
        else:
            print(f'Failed to upsert batch: {response.status_code}, {response.text}')
        
        # Clear the batch after sending
        batch_data = []

# Send any remaining rows in the last batch
if batch_data:
    response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
    
    if response.status_code in [200, 201]:
        print(f'Final batch of {len(batch_data)} rows upserted successfully')
    else:
        print(f'Failed to upsert final batch: {response.status_code}, {response.text}')


Final batch of 639 rows upserted successfully
